In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Pancreatic_Cancer/GSE212211'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Development of blood biomarkers that distinguish pancreatic cancer from biliary tract cancer"
!Series_summary	"We have developed a method for diagnosing pancreatic cancer and bile duct cancer based on miRNA expression information in the circulating blood."
!Series_summary	"2565 miRNAs in 426 serum samples were analyzed."
!Series_overall_design	"The design of this series was constructed in 257 hepatocellular carcinoma (HCC) patients and 41 cholangiocarcinoma carcinoma (ICC) patients and two gall bladder cancer patients."
Sample Characteristics Dictionary:
{0: ['disease/diagnosis: hepatocellular carcinoma'], 1: ['sample collection time: after operation beyand 14 days', 'sample collection time: before operation', 'sample collection time: after operation within 14 days', 'sample collection time: before operation at 2nd recurrence'], 2: ['molecule subtype: miRNA']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Provided dataset sample characteristics dictionary
sample_characteristics_dictionary = {
    0: ['disease/diagnosis: Cholangiocellular carcinoma', 
        'disease/diagnosis: hepatocellular carcinoma', 
        'disease/diagnosis: Gall bladder cancer'], 
    1: ['sample collection time: before operation', 
        'sample collection time: before operation at 2nd recurrence'], 
    2: ['molecule subtype: miRNA']
}

# Check gene expression data availability
# Since the dataset explicitly mentions miRNA data, we recognize this as not suitable
if any("miRNA" in item for sublist in sample_characteristics_dictionary.values() for item in sublist):
    is_gene_available = False

# Variable Conversion Doing Nothing since they are not available
def convert_trait(value):
    return None

def convert_age(value):
    match = re.search(r'(\d+)', value)
    return int(match.group(1)) if match else None

def convert_gender(value):
    if 'female' in value.lower():
        return 0
    elif 'male' in value.lower():
        return 1
    else:
        return None

# Additional check for completeness
for key, values in sample_characteristics_dictionary.items():
    if any('pancreatic cancer' in value.lower() for value in values):
        trait_row = key
        break  # Assume key is found if pancreatic cancer data is found

for key, values in sample_characteristics_dictionary.items():
    if any('age' in value.lower() for value in values):
        age_row = key
        break  # Assume only one key per variable

for key, values in sample_characteristics_dictionary.items():
    if any('gender' in value.lower() for value in values):
        gender_row = key
        break  # Assume only one key per variable

# Save Metadata
save_cohort_info('GSE212211', './preprocessed/Pancreatic_Cancer/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction - Skip as trait_row is None
